In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import spacy
import nltk

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download("punkt_tab")

!python -m spacy download es_core_news_sm

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erichuiza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/erichuiza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/erichuiza/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 43.0 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
nlp = spacy.load("es_core_news_sm")
stop_words = set(nltk.corpus.stopwords.words('spanish'))

In [4]:
def normalize_text(documents):
    """
    Normaliza una lista de documentos eliminando puntuación, convirtiendo a minúsculas,
    eliminando stopwords y lematizando.

    Args:
        documents (list): Una lista de cadenas de texto.

    Returns:
        list: Una lista de documentos normalizados.
    """
    normalized_list = []
    for doc in documents:
        # Pasa el documento completo por el pipeline de spaCy
        processed_doc = nlp(re.sub(r'[^\w\s]', '', doc.lower()))

        # Filtra tokens para lematizar, eliminando stopwords y no alfabéticos
        normalized_words = [
            token.lemma_ for token in processed_doc
            if token.is_alpha and token.text not in stop_words
        ]
        # Une las palabras normalizadas en una sola cadena
        normalized_list.append(" ".join(normalized_words))
    return normalized_list


In [5]:
# Documentos de entrada
corpus = [
    "El gato negro salta sobre la mesa.",
    "El perro salta sobre la cerca."
]

# Normaliza los documentos
normalized_corpus = normalize_text(corpus)
print("Documentos normalizados:\n", normalized_corpus)

# Implementa el modelo TF-IDF
# Crea una instancia de TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transforma los documentos normalizados en una matriz de valores TF-IDF
# fit_transform() aprende el vocabulario y crea la matriz en un solo paso
tfidf_matrix = vectorizer.fit_transform(normalized_corpus)

# Obtiene el vocabulario único
vocabulario = vectorizer.get_feature_names_out()
print("\nVocabulario (palabras únicas):\n", vocabulario)

# Convierte la matriz dispersa a un array denso para una mejor visualización
tfidf_array = tfidf_matrix.toarray()

# Muestra la matriz resultante en un DataFrame de Pandas para mayor claridad
df_tfidf = pd.DataFrame(tfidf_array, columns=vocabulario, index=[f"Documento {i+1}" for i in range(len(corpus))])
print("\nMatriz de TF-IDF (valores de importancia):\n")
print(df_tfidf)

Documentos normalizados:
 ['gato negro salta mesa', 'perro salta cerca']

Vocabulario (palabras únicas):
 ['cerca' 'gato' 'mesa' 'negro' 'perro' 'salta']

Matriz de TF-IDF (valores de importancia):

                cerca      gato      mesa     negro     perro     salta
Documento 1  0.000000  0.534046  0.534046  0.534046  0.000000  0.379978
Documento 2  0.631667  0.000000  0.000000  0.000000  0.631667  0.449436
